In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import re
import time

In [2]:
options = EdgeOptions()
options.use_chromium = True
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.set_capability("acceptInsecureCerts", True)
#Try to connect if the edgeriver version is 108
options.binary_location =r'C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe'

In [3]:
dicParfum={}
listeMarque=[]
listeIngredient=[]

In [4]:
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://www.sephora.fr/")
cookies=driver.find_element(By.XPATH,'//*[@id="footer_tc_privacy_button_2"]')
cookies.click()
for i in range (0,10):
    div=0
    driver.get("https://www.sephora.fr/parfum-meilleures-ventes/?srule=Sorting%20Rule%20-%20Best%20Sellers&start=0&sz=24")
    id=str(i+1)
    #print("top : "+ id )
    try:
        pathparfum="/html/body/div[2]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
        #//*[@id="search-result-items"]/li[1]/div
        Parfum=driver.find_element(By.XPATH, pathparfum)
    except :
        pathparfum="/html/body/div[1]/div[3]/div[2]/div[3]/div[2]/div[2]/ul/li["+id+"]"
        Parfum=driver.find_element(By.XPATH, pathparfum)
    if Parfum.get_attribute('class')=='grid-tile':
        marque=driver.find_element(By.XPATH,pathparfum+"/div/div[3]/div/a/span").get_attribute('innerText')
        #print(marque)
        listeMarque.append(marque)
        Parfum.click()
        #nom du parfum      
        nomParfum=driver.find_element(By.CLASS_NAME,'product-name.product-name-bold').get_attribute("innerText")
        #print(nomParfum)
        dicParfum[i]=nomParfum
        #liste des ingredients
        ingredient=driver.find_element(By.CLASS_NAME,'ingredients-content.tabs-panel').get_attribute('innerText')
        #print('Ingredient : '+ ingredient)
        listeIngredient.append(ingredient)
        #print("\n")
driver.close()
        

<ipython-input-4-47e148e7b33a>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)


# Gestion des Ingredients

In [5]:
phrase="Cette liste d'ingrédients peut faire l'objet de modifications, veuillez consulter l'emballage du produit acheté."
Ingredient=[]
for i in listeIngredient:
    ouisticram=i.replace("\n",'')
    ouisticram=ouisticram.replace(phrase,'')
    ousticram=ouisticram.replace(phrase,'')
    if "INGREDIENTS" in ouisticram:
        champenfeu=ouisticram.split(':')
        champenfeu=champenfeu[-1]
    else:
        champenfeu=ouisticram
    if "●" in champenfeu :
        simiabraz=champenfeu.split("●")
    elif " | " in champenfeu:
        simiabraz=champenfeu.split(" | ")
    elif ' ,' in champenfeu:
        simiabraz=champenfeu.split(" ,")
    elif ',' in champenfeu:
        simiabraz=champenfeu.split(",")
    elif " • " in champenfeu:
        simiabraz=champenfeu.split(" • ")
    else:
        print(simiabraz)
    Ingredient.append(simiabraz)
    

In [6]:
Ingredients=[]
for i in Ingredient:
    for j in i :
        if j.startswith('#'):
            ij=j.split(' ')
            j=ij[1]
        if j.startswith(' '):
            j=j[1:]
        if j.endswith(' '):
            j=j[:-1]
        Ingredients.append(j)

In [7]:
setIngredient=set(Ingredients)

In [8]:
descIng={}
Allergene=[]
endo=[]

In [9]:
%%time
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://incibeauty.com/ingredients")
cookies=driver.find_element(By.XPATH,"/html/body/div[6]/div[1]/div/div[2]/button")
cookies.click()
for i in setIngredient:
    print(i)
    driver.get("https://incibeauty.com/ingredients")
    search=driver.find_element(By.ID,'searchInci')
    search.click()
    search.clear()
    search.send_keys(i)
    count=0
    url=False
    b=True
    while b:
        count+=1
        search.send_keys(Keys.ENTER)
        if driver.current_url!="https://incibeauty.com/ingredients":
            b=False
            url=True
        if count>=10:
            b=False
    if url :
        listeInfo= WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'/html/body/section[1]/div/div/div/ul')))
        if 'Allergène' in listeInfo.get_attribute('innerText'):
            Allergene.append(i)
        if 'Perturbateur endocrinien' in listeInfo.get_attribute('innerText'):
            endo.append(i)
        try:
            aSavoir=driver.find_element(By.XPATH,'/html/body/section[1]/div/div/div/div[2]/div')
            descIng[i]=aSavoir.get_attribute('innerText')
        except:
            descIng[i]="Pas d'information supplémentaire"

driver.close()

<timed exec>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3


CI 60730 (EXT. VIOLET 2)
CI 14700 / RED 4
BHT
TRIS(TETRAMETHYLHYDROXYPIPERIDINOL) CITRATE
BENZYL BENZOATE
TOCOPHEROL
ISOEUGENOL
CI 15985 (YELLOW 6)
875178 50 : ALCOHOL
ETHYLHEXYL METHOXYCINNAMATE
CITRONELLOL
CITRUS AURANTIUM AMARA (BITTER ORANGE) FLOWER WATER
PENTAERYTHRITYL TETRA-DI-T-BUTYL HYDROXYHYDROCINNAMATE
CI 17200 / RED 33 (F.I.L. N292437/1).
AQUA (WATER)
POLYACRYLATE CROSSPOLYMER-6
ALCOHOL DENAT.
ALPHA-ISOMETHYL IONONE
PARFUM / FRAGRANCE
EVERNIA PRUNASTRI (OAK MOSS) EXTRACT
TRIETHYL CITRATE
PARFUM (FRAGRANCE)
LINALOOL
HYDROXYCITRONELLAL
FARNESOL
IL47-1
ANISE ALCOHOL
LIMONENE
CI 19140 (YELLOW 5)
BENZYL SALICYLATE
FARNESOLCITRAL
BENZYL ALCOHOL
CI 60730 / EXT. VIOLET 2 (F.I.L. C235192/1).
CITRAL
BENZYL CINNAMATE
T-BUTYL ALCOHOL
CINNAMYL ALCOHOL
AQUA / WATER / EAU
BUTYL METHOXYDIBENZOYLMETHANE
AQUA / WATER
IL60-1
ETHYLHEXYL SALICYLATE
CI 17200 (RED 33)
CI 14700 (RED 4)
CI 42090 (BLUE 1)
ALCOHOL
HEXYL CINNAMAL
CI 19140 / YELLOW 5
ALPHA,ISOMETHYL IONONE
METHYL ANTHRANILATE
DIETHYLAM

In [10]:
driver.quit()

In [11]:
Allergene

['ISOEUGENOL',
 'ALCOHOL DENAT.',
 'ALPHA-ISOMETHYL IONONE',
 'EVERNIA PRUNASTRI (OAK MOSS) EXTRACT',
 'LINALOOL',
 'FARNESOL',
 'ANISE ALCOHOL',
 'LIMONENE',
 'BENZYL ALCOHOL',
 'CITRAL',
 'BENZYL CINNAMATE',
 'ALPHA,ISOMETHYL IONONE',
 'EUGENOL',
 'GERANIOL']

In [12]:
descIng

{'CI 60730 (EXT. VIOLET 2)': 'À SAVOIR\nL\'acide citrique est un des principaux actifs du citron. Il est souvent utilisé pour équilibrer le pH (trop basique) des produits cosmétiques. Il est aussi présent dans certains produits de bain (bombes de bain, galets de bain ou "poudres magiques") en raison de ses propriétés effervescentes.',
 'CI 14700 / RED 4': 'À SAVOIR\nL\'acide citrique est un des principaux actifs du citron. Il est souvent utilisé pour équilibrer le pH (trop basique) des produits cosmétiques. Il est aussi présent dans certains produits de bain (bombes de bain, galets de bain ou "poudres magiques") en raison de ses propriétés effervescentes.',
 'BHT': 'À SAVOIR\nLe BHT ou Butylhydroxytoluène est un phénol qui se présente sous la forme de cristaux blanc jaunâtre. Il est employé pour ses propriétés antioxydantes en cosmétique.\nLe CIRC (Centre international de recherche sur le cancer) classe le BHT parmi les substances dont l\'innocuité n\'a pas pu être établie, l\'ARTAC (A

# Gestion des marques

In [13]:
marqueEco={listeMarque[i]+' ': 'Sans engagement' for i in range(len(listeMarque))}
marqueEco

{'GIVENCHY ': 'Sans engagement',
 'DIOR ': 'Sans engagement',
 'YVES SAINT LAURENT ': 'Sans engagement',
 'CHANEL ': 'Sans engagement',
 'PACO RABANNE ': 'Sans engagement',
 'PRADA ': 'Sans engagement'}

In [24]:
%%time
driver = Edge(executable_path =r"C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe", options = options)
driver.get("https://sciencebasedtargets.org/companies-taking-action#dashboard")
time.sleep(5)
cookies=driver.find_element(By.CLASS_NAME,"font-bold.text-center.rounded-full.appearance-none.o-btn.border-3.border-currentColor.flex-shrink-0.is-current")
cookies.click()
perpage=Select(driver.find_element(By.XPATH,'//*[@id="per-page"]'))
perpage.select_by_value("50")
while True:
    #try:
        for ids in range (1,11):
            #select la ligne du tableau
            table=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[1]/div")
            driver.execute_script("window.scrollTo(0, 10)")
            Marque=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[1]/div/div[1]/div[1]").get_attribute('innerText')

            b=True
            #Click sur view more si visible (sinon c'est que deja cliqué)
            while b:
                count+=1
                driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/label").click()
                if driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/div/div[1]/div[4]").is_displayed():
                    b=False
                if count>=10:
                    b=False
            sectors=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/div/div[1]/div[4]")
            try:
                target=driver.find_element(By.XPATH,"/html/body/main/div/div[3]/div[2]/div/div/div/div[2]/div[2]/div["+str(ids)+"]/div[2]/div/div[3]/div[2]").get_attribute('innerText')
            except : 
                target = "Commited with no specifics"
            if sectors in ['Consumer Durables, Household and Personal Products','Textiles, Apparel, Footwear and Luxury Goods']:
                marqueEco[Marque]=target.get_attribute('innerText')
                print(target)
        myElem=driver.find_element(By.XPATH,'/html/body/main/div/div[3]/div[2]/div/div/div/div[3]/div/button[2]')
        myElem.click()
    #except:
        #break

<timed exec>:1: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3


Containers and Packaging
Professional Services
Trading Companies and Distributors, and Commercial Services and Supplies
Food and Beverage Processing
Trading Companies and Distributors, and Commercial Services and Supplies
Telecommunication Services
Software and Services
Consumer Durables, Household and Personal Products
Banks, Diverse Financials, Insurance
Chemicals
Pharmaceuticals, Biotechnology and Life
Pharmaceuticals, Biotechnology and Life
Electrical Equipment and Machinery
Retailing
Banks, Diverse Financials, Insurance
Ground Transportation - Highways and Railtracks
Technology Hardware and Equipment
Trading Companies and Distributors, and Commercial Services and Supplies
Trading Companies and Distributors, and Commercial Services and Supplies
Retailing
Software and Services
Electrical Equipment and Machinery
Professional Services
Banks, Diverse Financials, Insurance
Consumer Durables, Household and Personal Products
Professional Services
Real Estate
Pharmaceuticals, Biotechnology

Semiconductors and Semiconductors Equipment
Hotels, Restaurants and Leisure, and Tourism Services
Food Production - Agricultural Production
Electrical Equipment and Machinery
Electrical Equipment and Machinery
Food and Beverage Processing
Food and Beverage Processing
Hotels, Restaurants and Leisure, and Tourism Services
Food and Beverage Processing
Real Estate
Trading Companies and Distributors, and Commercial Services and Supplies
Trading Companies and Distributors, and Commercial Services and Supplies
Real Estate
Telecommunication Services
Ground Transportation - Trucking Transportation
Food Production - Agricultural Production
Textiles, Apparel, Footwear and Luxury Goods
Telecommunication Services
Professional Services
Software and Services
Professional Services
Water Transportation - Water Transportation
Electrical Equipment and Machinery
Forest and Paper Products - Forestry, Timber, Pulp and Paper, Rubber
Professional Services
Electrical Equipment and Machinery
Banks, Diverse Fina

Software and Services
Trading Companies and Distributors, and Commercial Services and Supplies
Electrical Equipment and Machinery
Containers and Packaging
Automobiles and Components
Textiles, Apparel, Footwear and Luxury Goods
Banks, Diverse Financials, Insurance
Hotels, Restaurants and Leisure, and Tourism Services
Professional Services
Trading Companies and Distributors, and Commercial Services and Supplies
Professional Services
Chemicals
Construction and Engineering
Ground Transportation - Railroads Transportation
Construction Materials
Building Products
Healthcare Equipment and Supplies
Construction and Engineering
Professional Services
Healthcare Equipment and Supplies
Automobiles and Components
Technology Hardware and Equipment
Healthcare Equipment and Supplies
Ground Transportation - Railroads Transportation
Professional Services
Professional Services
Real Estate
Technology Hardware and Equipment
Consumer Durables, Household and Personal Products
Software and Services
Constructi

Retailing
Textiles, Apparel, Footwear and Luxury Goods
Food and Beverage Processing
Building Products
Software and Services
Food and Beverage Processing
Air Freight Transportation and Logistics
Building Products
Electrical Equipment and Machinery
Textiles, Apparel, Footwear and Luxury Goods
Retailing
Professional Services
Automobiles and Components
Consumer Durables, Household and Personal Products
Electrical Equipment and Machinery
Professional Services
Air Freight Transportation and Logistics
Automobiles and Components
Food and Beverage Processing
Electrical Equipment and Machinery
Software and Services
Ground Transportation - Trucking Transportation
Specialized Consumer Services
Food and Beverage Processing
Tobacco
Technology Hardware and Equipment
Professional Services
Air Freight Transportation and Logistics
Technology Hardware and Equipment
Media
Containers and Packaging
Food and Beverage Processing
Textiles, Apparel, Footwear and Luxury Goods
Technology Hardware and Equipment
Fo

Professional Services
Software and Services
Consumer Durables, Household and Personal Products
Pharmaceuticals, Biotechnology and Life
Textiles, Apparel, Footwear and Luxury Goods
Textiles, Apparel, Footwear and Luxury Goods
Semiconductors and Semiconductors Equipment
Specialized Consumer Services
Consumer Durables, Household and Personal Products
Software and Services
Food and Beverage Processing
Electric Utilities and Independent Power Producers and Energy Traders (including fossil, alternative and nuclear energy)
Professional Services
Consumer Durables, Household and Personal Products
Food and Beverage Processing
Mining - Iron, Aluminum, Other Metals
Containers and Packaging
Banks, Diverse Financials, Insurance
Professional Services
Construction and Engineering
Professional Services
Chemicals
Trading Companies and Distributors, and Commercial Services and Supplies
Containers and Packaging
Hotels, Restaurants and Leisure, and Tourism Services
Professional Services
Software and Servic

Consumer Durables, Household and Personal Products
Forest and Paper Products - Forestry, Timber, Pulp and Paper, Rubber
Consumer Durables, Household and Personal Products
Forest and Paper Products - Forestry, Timber, Pulp and Paper, Rubber
Healthcare Equipment and Supplies
Consumer Durables, Household and Personal Products
Textiles, Apparel, Footwear and Luxury Goods
Software and Services
Food Production - Agricultural Production
Retailing
Professional Services
Textiles, Apparel, Footwear and Luxury Goods
Media
Software and Services
Professional Services
Consumer Durables, Household and Personal Products
Professional Services
Chemicals
Professional Services
Specialized Financial Services, Consumer Finance, Insurance Brokerage Firms
Professional Services
Textiles, Apparel, Footwear and Luxury Goods
Containers and Packaging
Textiles, Apparel, Footwear and Luxury Goods
Food and Beverage Processing
Professional Services
Mining - Iron, Aluminum, Other Metals
Solid Waste Management Utilities

Containers and Packaging
Software and Services
Trading Companies and Distributors, and Commercial Services and Supplies
Telecommunication Services
Ground Transportation - Highways and Railtracks
Technology Hardware and Equipment
Professional Services
Software and Services


ElementNotInteractableException: Message: element not interactable
  (Session info: MicrosoftEdge=120.0.2210.121)


In [18]:
marqueEco

{'GIVENCHY ': 'Sans engagement',
 'DIOR ': 'Sans engagement',
 'YVES SAINT LAURENT ': 'Sans engagement',
 'CHANEL ': 'Sans engagement',
 'PACO RABANNE ': 'Sans engagement',
 'PRADA ': 'Sans engagement'}

In [25]:
marqueEco

{'GIVENCHY ': 'Sans engagement',
 'DIOR ': 'Sans engagement',
 'YVES SAINT LAURENT ': 'Sans engagement',
 'CHANEL ': 'Sans engagement',
 'PACO RABANNE ': 'Sans engagement',
 'PRADA ': 'Sans engagement'}